In [91]:
import re
import bisect
bisect.bisect_left([10, 30, 40],  41)

re.split('[^A-Za-z]+', 'ala ma kota')


['ala', 'ma', 'kota']

In [92]:
rdd_aow = sc.textFile('data/artwar.1b.txt')

In [93]:
import re
def is_heading(s):
    return re.match(r'^[IVX]+\..*', s) or s.startswith("THE END")

chapters = rdd_aow.zipWithIndex() \
    .filter(lambda (s,i): is_heading(s)).collect()
    
display(chapters)

[(u'I. Laying Plans', 15),
 (u'II. Waging War', 106),
 (u'III. Attack by Stratagem', 184),
 (u'IV. Tactical Dispositions', 266),
 (u'V. Energy', 337),
 (u'VI. Weak Points and Strong', 426),
 (u'VII. Maneuvering', 562),
 (u'VIII. Variation in Tactics', 692),
 (u'IX. The Army on the March', 748),
 (u'X. Terrain', 908),
 (u'XI. The Nine Situations', 1033),
 (u'XII. The Attack by Fire', 1287),
 (u'XIII. The Use of Spies', 1365),
 (u'THE END', 1467)]

In [94]:
import operator
from itertools import islice


chapter_bounds = sc.broadcast([ i for (title,i) in chapters])

rdd_tmp = rdd_aow.zipWithIndex() \
    .filter(lambda (s,i): s and not is_heading(s)) \
    .map(lambda (s,i): (bisect.bisect_left(chapter_bounds.value, i),s)) \
    .flatMap(lambda (ch,line): ((ch, word) for word in re.split('[^a-z\']+', line.lower()) if word)) \
    .map(lambda chAndWord: (chAndWord, 1)) \
    .reduceByKey(operator.add) \
    .map(lambda ((ch, word), count): (ch, (word, count))) \
    .sortBy(lambda (ch, (word, count)): count, False) \
    .groupByKey() \
    .map(lambda (ch, it): (ch, list(islice(it,10))))

In [95]:
rdd_tmp.collect()

[(0,
  [(u'font', 32),
   (u'color', 17),
   (u'b', 12),
   (u'the', 8),
   (u'a', 8),
   (u'tzu', 6),
   (u'c', 6),
   (u'classics', 6),
   (u'http', 5),
   (u'edu', 5)]),
 (2,
  [(u'the', 38),
   (u'of', 26),
   (u'and', 18),
   (u'be', 17),
   (u'to', 16),
   (u'a', 12),
   (u'will', 12),
   (u'is', 11),
   (u'in', 11),
   (u'your', 7)]),
 (4,
  [(u'the', 37),
   (u'of', 25),
   (u'is', 19),
   (u'to', 19),
   (u'a', 12),
   (u'and', 9),
   (u'in', 9),
   (u'enemy', 7),
   (u'defeat', 7),
   (u'victory', 7)]),
 (6,
  [(u'the', 69),
   (u'to', 39),
   (u'be', 26),
   (u'and', 23),
   (u'in', 21),
   (u'of', 21),
   (u'he', 19),
   (u'his', 19),
   (u'is', 18),
   (u'can', 17)]),
 (8,
  [(u'the', 25),
   (u'of', 19),
   (u'to', 14),
   (u'be', 13),
   (u'not', 12),
   (u'in', 12),
   (u'which', 11),
   (u'his', 9),
   (u'and', 7),
   (u'must', 7)]),
 (10,
  [(u'the', 60),
   (u'to', 31),
   (u'and', 30),
   (u'is', 29),
   (u'a', 21),
   (u'of', 20),
   (u'are', 19),
   (u'you', 17),


In [96]:
chapterCounts = rdd_aow.zipWithIndex() \
    .filter(lambda (s,i): s and not is_heading(s)) \
    .map(lambda (s,i): (bisect.bisect_left(chapter_bounds.value, i),s)) \
    .flatMap(lambda (ch,line): ((word, ch) for word in re.split('[^a-z\']+', line.lower()) if word)) \
    .distinct() \
    .countByKey()

print(len(chapters))    
sorted(chapterCounts.items(), key=lambda kv:kv[1], reverse=True)[0:20]

14


[(u'is', 15),
 (u'the', 15),
 (u'by', 15),
 (u'of', 15),
 (u'that', 14),
 (u'tzu', 14),
 (u'in', 14),
 (u'for', 14),
 (u'may', 14),
 (u'and', 14),
 (u'with', 14),
 (u'to', 14),
 (u'a', 14),
 (u'its', 14),
 (u'sun', 14),
 (u'army', 13),
 (u'said', 13),
 (u'if', 13),
 (u'not', 13),
 (u'as', 13)]

In [97]:
from math import log
idf = dict( (w, log(14.0/c)) for w,c in chapterCounts.items())
sorted(idf.items(), key=lambda kv:kv[1], reverse=True)[0:20]

[(u'secondly', 2.6390573296152584),
 (u'dynasty', 2.6390573296152584),
 (u'woods', 2.6390573296152584),
 (u'aides', 2.6390573296152584),
 (u'rampart', 2.6390573296152584),
 (u'captain', 2.6390573296152584),
 (u'saved', 2.6390573296152584),
 (u'leeward', 2.6390573296152584),
 (u'applied', 2.6390573296152584),
 (u'lure', 2.6390573296152584),
 (u'govern', 2.6390573296152584),
 (u'affect', 2.6390573296152584),
 (u'bringing', 2.6390573296152584),
 (u'bewildered', 2.6390573296152584),
 (u'piling', 2.6390573296152584),
 (u'heading', 2.6390573296152584),
 (u'triumph', 2.6390573296152584),
 (u'hanging', 2.6390573296152584),
 (u'aim', 2.6390573296152584),
 (u'deliberate', 2.6390573296152584)]

In [98]:
idfBR = sc.broadcast(idf)

rddTD_IDF = rdd_aow.zipWithIndex() \
    .filter(lambda (s,i): s and not is_heading(s)) \
    .map(lambda (s,i): (bisect.bisect_left(chapter_bounds.value, i),s)) \
    .flatMap(lambda (ch,line): ((ch, word) for word in re.split('[^a-z\']+', line.lower()) if word)) \
    .map(lambda chAndWord: (chAndWord, 1)) \
    .reduceByKey(operator.add) \
    .map(lambda ((ch, word), count): (ch, (word, count * idfBR.value[word]))) \
    .sortBy(lambda (ch, (word, count)): count, False) \
    .groupByKey() \
    .map(lambda (ch, it): (ch, list(islice(it,20))))



In [99]:
rddTD_IDF.collect()

[(0,
  [(u'font', 84.44983454768827),
   (u'color', 44.86397460345939),
   (u'b', 31.6686879553831),
   (u'http', 13.195286648076292),
   (u'artwar', 13.195286648076292),
   (u'c', 11.675460894331879),
   (u'classics', 11.675460894331879),
   (u'g', 10.556229318461034),
   (u'tr', 10.556229318461034),
   (u'td', 10.556229318461034),
   (u'br', 10.556229318461034),
   (u'table', 10.556229318461034),
   (u'f', 10.556229318461034),
   (u'page', 10.556229318461034),
   (u'href', 10.556229318461034),
   (u'txt', 10.556229318461034),
   (u'edu', 9.729550745276565),
   (u'mit', 9.729550745276565),
   (u'o', 7.783640596221253),
   (u'html', 5.8377304471659395)]),
 (2,
  [(u'chariots', 11.675460894331879),
   (u'substance', 7.917171988845775),
   (u'thousand', 6.161780163788596),
   (u'weapons', 5.278114659230517),
   (u'prices', 5.278114659230517),
   (u'equivalent', 5.278114659230517),
   (u'drained', 5.278114659230517),
   (u'tenths', 5.278114659230517),
   (u'damped', 5.278114659230517),
  